In [14]:
import json
import re
from limpiador_texto import LimpiezaTexto

class TextoJson:
    def __init__(self, nombreArchivo):
        self.limpiador = LimpiezaTexto(nombreArchivo)
        self.informacion = self.limpiador.obtener_texto_procesado()
        # vectores
        self.data = {
            "antecedentes_personales": {},
            "antecedentes_familiares": {},
            "estadia_tumoral": {},
            "inmunohistoquímica_tumoral": {},
            "tipo": {},
        }

    def estructurar_data(self):
        return json.dumps(self.data, indent=4, ensure_ascii=False)
    
    def imprimir_info(self):
        for linea in self.informacion:
            print(linea)
    
    def imprimir_data(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))

    def extract_units(self, data):
        # Regular expression to match patterns like "años", "meses", "días", "horas"
        unit_pattern = re.compile(
            r'\b(\d+\.?\d*)\s*(año|años|mes|meses|día|días|hora|horas)\b|\b(año|años|mes|meses|día|días|hora|horas)\s*(\d+\.?\d*)\b',
            re.IGNORECASE
        )
        matches = unit_pattern.findall(data)
        
        units = []
        for match in matches:
            if match[0] and match[1]:  # Caso: "1 año" o "1.5 años"
                value = float(match[0]) if '.' in match[0] else int(match[0])  # Parseo de dígitos
                units.append((value, match[1].lower()))
            elif match[2] and match[3]:  # Caso: "año 1" o "años 1.5"
                value = float(match[3]) if '.' in match[3] else int(match[3])  # Parseo de dígitos
                units.append((value, match[2].lower()))
        
        return units
    
    def extraer_datos_antecedentes_personales(self):
        #imprimimos las lineas del self.informacion
        for linea in self.informacion:
            print(linea)
        # secciones de antecedentes_personales
        antecedentes_personales_secciones = {
            "edad": None,
            "sexo": "femenino",
            "peso": None,
            "talla": None,
            "preferencia": None,
            "índice_tabáquico": {
                "fuma": None,
                "observaciones": None
            },
            "alcohol": None,
            "drogas": None,
            "comorbilidades": None,
            "antecedentes_ginecológicos": {
                "fum": None,
                "menarca": None,
                "embarazos": None,
                "partos": None,
                "trh": None,
                "estado_hormonal": None,
                "métodos_anticonceptivos": None
            }
        }

        #palaras clave para antecedentes_personales que buscaremos en el texto
        antecedentes_personales_keywords = [
            "edad", "sexo", "peso", "talla", "preferencia", 
            "índice tabáquico", "tabaco", "tabaquismo", "alcohol", "drogas", 
            "comorbilidades", "antecedentes ginecológicos", "menarca", "embarazos", "partos", "fum", "trh", 
            "estado hormonal", "métodos anticonceptivos"
        ]

        texto_procesado = self.informacion
        isEdad = False

        # Agregar una regex para detectar edades en el formato "XX años"
        edad_regex = re.compile(
            r'\b(?:edad:?\s*(\d+)(?:\s*años)?)|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(\w+)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$|(?:\b(\d+)\s*años(?:\s*de\s*edad)?)|^(\w+)\s*/\s*(.+?)\s*/\s*(\d+\s*años(?:\s*y\s*\d+\s*meses)?)\s*/\s*(.*)$',re.IGNORECASE
        )
        for linea in texto_procesado:
            # Buscar una edad en el formato específico
            match = edad_regex.match(linea)
            if match:
                edad_valor = linea
                
                if edad_valor:
                    try:
                        unidades = self.extract_units(edad_valor)
                        edad_dict = {}
                        for cantidad, unidad in unidades:
                            if unidad == "años":
                                edad_dict["años"] = cantidad
                            elif unidad == "meses":
                                edad_dict["meses"] = cantidad
                            elif unidad == "días":
                                edad_dict["días"] = cantidad
                            elif unidad == "horas":
                                edad_dict["horas"] = cantidad
                        antecedentes_personales_secciones["edad"] = edad_dict
                    except (IndexError, ValueError):
                        antecedentes_personales_secciones["edad"] = {"años": None}
                
                isEdad = True
            
            for keyword in antecedentes_personales_keywords:
                if keyword in linea:
                    
                    # Si la keyword es "tabaquismo" o "tabaco" o "índice tabáquico" y si tiene o no ":" que separan
                    if keyword == "tabaquismo" or keyword == "tabaco" or keyword == "índice tabáquico":
                        # Removemos los ":"
                        if ":" in linea:
                            linea = linea.replace(":", "")
                        
                        # Extraemos lo que sea diferente a "negado" y guardamos el resto en una variable
                        if "negado" in linea:
                            antecedentes_personales_secciones["índice_tabáquico"]["fuma"] = "negado"
                            # Verificamos si hay algo más en la línea
                            if len(linea.split("negado")[1].strip()) > 0:
                                notas = linea.split("negado")[1].strip()
                                observaciones = notas.split(". ")
                                observaciones_list = []
                                for observacion in observaciones:
                                    unidades = self.extract_units(observacion)
                                    if unidades:
                                        exposicion = {"tipo": observacion, "tiempo": {}}
                                        for cantidad, unidad in unidades:
                                            if unidad == "años":
                                                exposicion["tiempo"]["años"] = cantidad
                                            elif unidad == "meses":
                                                exposicion["tiempo"]["meses"] = cantidad
                                            elif unidad == "días":
                                                exposicion["tiempo"]["días"] = cantidad
                                            elif unidad == "horas":
                                                exposicion["tiempo"]["horas"] = cantidad
                                        observaciones_list.append(exposicion)
                                
                                if observaciones_list:
                                    antecedentes_personales_secciones["índice_tabáquico"]["observaciones"] = observaciones_list
                                else:
                                    antecedentes_personales_secciones["índice_tabáquico"]["observaciones"] = None

                    elif keyword in ["fum", "menarca", "trh", "estado hormonal", "métodos anticonceptivos", "embarazos", "partos"]:
                        # Verificar las keywords de antecedentes ginecológicos en la misma línea
                        pattern = re.compile(r'(fum|menarca|trh|estado hormonal|métodos anticonceptivos|embarazos|partos)', re.IGNORECASE)
                        matches = pattern.findall(linea)
                        if len(matches) > 1:
                            # Dividir la línea en partes basadas en varios separadores posibles
                            partes = re.split(r'[\/,\.]', linea)
                            for parte in partes:
                                # Asegurarte de que cada elemento sea una cadena válida
                                if isinstance(parte, str):  # Por seguridad, verifica si el elemento es una cadena
                                    parte = parte.strip()  # Elimina espacios al inicio y al final
                                    
                                for keyword in matches:
                                    if keyword in parte:
                                        valor = None
                                        if ":" in parte:
                                            valor = parte.split(":")[1].strip()
                                        else:
                                            # Verificar si el valor precede a la clave
                                            if parte.strip().startswith(keyword):
                                                valor = parte.split(keyword)[1].strip()
                                            else:
                                                valor = parte.split(keyword)[0].strip()
                                        
                                        # Usar extract_units para procesar el valor
                                        try:
                                            unidades = self.extract_units(valor)
                                            if unidades:
                                                valor_dict = {}
                                                for cantidad, unidad in unidades:
                                                    if unidad == "años":
                                                        valor_dict["años"] = cantidad
                                                    elif unidad == "meses":
                                                        valor_dict["meses"] = cantidad
                                                    elif unidad == "días":
                                                        valor_dict["días"] = cantidad
                                                    elif unidad == "horas":
                                                        valor_dict["horas"] = cantidad
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = valor_dict
                                            else:
                                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = int(valor)
                                        except (IndexError, ValueError):
                                            antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = valor
                        else:
                            # Procesar la línea si solo hay una coincidencia
                            keyword = matches[0]
                            valor = None
                            if ":" in linea:
                                valor = linea.split(":")[1].strip()
                            else:
                                valor = linea.split(keyword)[1].strip()
                            
                            # Usar extract_units para procesar el valor
                            try:
                                unidades = self.extract_units(valor)
                                if unidades:
                                    valor_dict = {}
                                    for cantidad, unidad in unidades:
                                        if unidad == "años":
                                            valor_dict["años"] = cantidad
                                        elif unidad == "meses":
                                            valor_dict["meses"] = cantidad
                                        elif unidad == "días":
                                            valor_dict["días"] = cantidad
                                        elif unidad == "horas":
                                            valor_dict["horas"] = cantidad
                                    antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = valor_dict
                                else:
                                    antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = valor
                            except (IndexError, ValueError):
                                antecedentes_personales_secciones["antecedentes_ginecológicos"][keyword.strip().replace(" ", "_")] = valor
                    
                    ##tratamos las comoorbilidades dado que es una linea con varias comorbilidades separadas por "/"
                    elif keyword == "comorbilidades":
                        comorbilidades = linea
                        # Dividir la cadena en comorbilidades individuales
                        comorbilidades_list = re.split(r'\d+\.\s', comorbilidades)

                        # Eliminar posibles cadenas vacías resultantes de la división
                        comorbilidades_list = [comorbilidad for comorbilidad in comorbilidades_list if comorbilidad]

                        comorbilidades_dict = {}

                        for comorbilidad in comorbilidades_list:
                            comorbilidad = comorbilidad.strip()
                            # Extraer el año
                            match = re.search(r'\((\d{4})\)', comorbilidad)
                            if match:
                                year = match.group(1)
                                # Separar nombre y descripción
                                parts = comorbilidad.split(f"({year})")
                                nombre = parts[0].strip()
                                descripcion = parts[1].strip() if len(parts) > 1 else ""

                                # Si la descripción es solo un punto, lo agregamos al nombre
                                if descripcion == ".":
                                    descripcion = None  # Dejamos la descripción vacía
                                    nombre = nombre.rstrip('.')  # Quitamos el punto del nombre

                                if year not in comorbilidades_dict:
                                    comorbilidades_dict[year] = []

                                comorbilidades_dict[year].append({
                                    "nombre": nombre.replace("comorbilidades: ", "").strip(),
                                    "descripción": descripcion.lstrip(': ') if descripcion else None
                                })
                        antecedentes_personales_secciones["comorbilidades"] = comorbilidades_dict

                    else:
                        if keyword == "edad" and isEdad == True:
                            continue
                        elif ":" in linea:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(":")[1].strip()
                        
                        else:
                            antecedentes_personales_secciones[keyword.replace(" ", "_")] = linea.split(keyword)[1].strip()

        self.data["antecedentes_personales"] = antecedentes_personales_secciones
        #self.imprimir_data()

    def extraer_datos_inmunohistoquuímica_tumoral(self):
        #imprimimos las lineas del self.informacion
        for linea in self.informacion:
            print(linea)

        # secciones de inmunohistoquímica_tumoral
        inmunohistoquímica_tumoral_secciones = {
            "mama_izquierda": {
                "re": None,
                "rp": None,
                "her2": None,
                "ki67": None,
                "gh": None
            }, 
            "mama_derecha": {
                "re": None,
                "rp": None,
                "her2": None,
                "ki67": None,
                "gh": None
            }
        }

        for linea in self.informacion:
            linea = linea.strip()

            # Extraer biología tumoral
            if "biología tumoral" in linea:
                match = re.search(r"mama izquierda:(.+?)mama derecha:(.+)", linea)
                if match:
                    izquierda = match.group(1).strip()
                    derecha = match.group(2).strip()

                    # Extraer datos específicos de cada mama
                    for mama, texto in [("mama_izquierda", izquierda), ("mama_derecha", derecha)]:
                        inmunohistoquímica_tumoral_secciones[mama]["re"] = re.search(r"re (\d+%)", texto).group(1) if re.search(r"re (\d+%)", texto) else None
                        inmunohistoquímica_tumoral_secciones[mama]["rp"] = re.search(r"rp (\d+%)", texto).group(1) if re.search(r"rp (\d+%)", texto) else None
                        inmunohistoquímica_tumoral_secciones[mama]["her2"] = "negativo" if "her2 negativo" in texto else "positivo"
                        inmunohistoquímica_tumoral_secciones[mama]["ki67"] = re.search(r"ki67 (\d+%)", texto).group(1) if re.search(r"ki67 (\d+%)", texto) else None
                        inmunohistoquímica_tumoral_secciones[mama]["gh"] = re.search(r"g\d", texto).group(0) if re.search(r"g\d", texto) else None

           

        self.data["inmunohistoquímica_tumoral"] = inmunohistoquímica_tumoral_secciones
        self.imprimir_data()

        
    

procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido6.txt')
procesador.extraer_datos_antecedentes_personales()
procesador.extraer_datos_inmunohistoquuímica_tumoral()

mlaa / 314262 / 86 años / dr. soto
comorbilidades: hipertensión arterial sistémica (1995).
fum: 47 años
menarca a los 14 años. 2 embarazos, 2 partos.
métodos anticonceptivos desconoce.
cirugías: apendicectomía a los 20 años.
tabaquismo negado: exposición al humo de leña por 40 años por 2 horas.
ahf oncológicos: negados
originaria y residente: puebla
seguridad social: 1 (imss)
ocupación: hogar
cáncer de mama bilateral. mi cd! pt1c, pn1a, mo / md cli pt2, cno mo rh+ iher2-
extensión del tumor: mama izquierda ec ja (pt1c, pn1a, mo) mama derecha ec 1la (pt2, no, mo)
biología tumoral: mama izquierda: g3, re 90%, rp 90%, her2 negativo, ki67 30% mama derecha: g2, re 95%, rp 95%, her2 negativo, ki67 5%
estado hormonal: postmenopáusica
mlaa / 314262 / 86 años / dr. soto
comorbilidades: hipertensión arterial sistémica (1995).
fum: 47 años
menarca a los 14 años. 2 embarazos, 2 partos.
métodos anticonceptivos desconoce.
cirugías: apendicectomía a los 20 años.
tabaquismo negado: exposición al humo 

<>:322: SyntaxWarning: invalid escape sequence '\D'
<>:322: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_2164\863321943.py:322: SyntaxWarning: invalid escape sequence '\D'
  procesador = TextoJson('D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\ExtraccionTexto\TextoExtraido6.txt')
